# Calendar View

Given a department code and a term, creates a calendar view of scheduled courses in Banner for the purpose of visualizing course scheduling conflicts.

The primary use case for this is determining where to schedule a new IIS class so as to minimize conflicts with other departments, but it would probably be equally useful for a department to visualize its own schedule.

Note that you will need to [install and reference NuGet packages](https://docs.microsoft.com/en-us/nuget/tools/nuget-exe-cli-reference) as appropriate on your system.

## Common code

The code below describes:

- A basic Banner POST for all courses offered by a department 
- A basic Banner POST for a specific class
- XPath oriented methods for processing the html (from HtmlAgilityPack)
- Html autorepair methods for fixing bad html that breaks XPath (from AngleSharp)
- General functions for extracting calendar data and ICS format

Since AngleSharp does not support XPath, we use two libraries for html manipulation.
Refactoring to remove XPath/HtmlAgilityPack would probably be sensible at some point.

In [33]:
#r "/z/aolney/repos/FSharp.Data.2.3.2/lib/net40/FSharp.Data.dll"
#r "/z/aolney/repos/HtmlAgilityPack.1.4.9.5/lib/Net40/HtmlAgilityPack.dll"
#r "/z/aolney/repos/AngleSharp.0.9.10/lib/net45/AngleSharp.dll"

open FSharp.Data
open HtmlAgilityPack
open System

let QueryClass (dept:string) (number:string) (term:string) =
    let result = 
        Http.RequestString( 
            "https://banssbprod.memphis.edu/pls/PROD/bwckschd.p_get_crse_unsec",
            query=[
                "term_in", term.Trim();
                "sel_subj","dummy";
                "sel_day","dummy";
                "sel_schd","dummy";
                "sel_insm","dummy";
                "sel_camp","dummy";
                "sel_levl","dummy";
                "sel_sess","dummy";
                "sel_instr","dummy";
                "sel_ptrm","dummy";
                "sel_attr","dummy";
                "sel_subj",dept.Trim();
                "sel_crse",number.Trim();
                "sel_title","";
                "sel_insm","%";
                "sel_from_cred","";
                "sel_to_cred","";
                "sel_camp","%";
                "sel_levl","%";
                "sel_ptrm","%";
                "sel_instr","%";
                "sel_attr","%";
                "begin_hh","0";
                "begin_mi","0";
                "begin_ap","a";
                "end_hh","0";
                "end_mi","0";
                "end_ap","a" ], 
            httpMethod="POST")
    let classFound = result.Contains("No classes were found that meet your search criteria") |> not
    //
    classFound,result

let QueryDepartment (dept:string) (term:string) =
    QueryClass dept "" term |> snd

//Creates a doc from html string
let createDoc html =
    let doc = new HtmlDocument()
    doc.LoadHtml html
    doc.DocumentNode
   
let descendants (name : string) (node : HtmlNode) = 
    node.Descendants name
    
let selectNodes (query : string) (node : HtmlNode) = 
    node.SelectNodes query

let inline innerText (node : HtmlNode) = 
    node.InnerText

let inline attr name (node : HtmlNode) = 
    node.GetAttributeValue(name, "")

let inline hasAttr name value node = 
    attr name node = value
    
//Use AngleSharp to auto-fix broken documents; converts to HTML5 standard
let AutoFixHtml (html : string) =
    let parser = new AngleSharp.Parser.Html.HtmlParser();
    let document = parser.Parse( html );
    document.DocumentElement.OuterHtml

let createDocAutoFix html =
    let doc = new HtmlDocument()
    html |> AutoFixHtml |> doc.LoadHtml
    doc.DocumentNode

//Extraction data from HTML
type Class =
    {
        Title: string
        Type: string
        Time: string
        StartDateTime: DateTime //hackish: date and time components have slightly different semantics
        EndDateTime: DateTime //hackish: date and time components have slightly different semantics
        Days: string[]
        Where: string
        DateRange: string
        ScheduleType: string
        Instructors: string
    }

let GetDays dayString = 
    if dayString = "&nbsp;" then
        [||]
    else 
        let result = new ResizeArray<string>()
        for c in dayString do
            match c with
            | 'M' -> result.Add( "MO" )
            | 'T' -> result.Add( "TU" )
            | 'W' -> result.Add( "WE" )
            | 'R' -> result.Add( "TH" )
            | 'F' -> result.Add( "FR" )
            | 'S' -> result.Add( "SA" )
            | _ -> ()
        //
        result.ToArray()
        
let GetStartEndDateTime (dateNode:HtmlNode) (timeNode:HtmlNode) = 
    let dateSplit = dateNode.InnerHtml.Split('-')
    let timeSplit = timeNode.InnerHtml.Split('-')
    match dateSplit.Length,timeSplit.Length with
    //we have valid dates and times
    | 2,2 -> 
        let s = dateSplit.[0].Trim() + " " + timeSplit.[0].Trim() |> DateTime.Parse
        let e = dateSplit.[1].Trim() + " " + timeSplit.[1].Trim() |> DateTime.Parse
        s,e
    //all other cases we can't put on calendar; we use MinValue instead of options
    | 2,_ -> DateTime.MinValue,DateTime.MinValue 
    | _,2 -> DateTime.MinValue,DateTime.MinValue 
    | _,_ -> DateTime.MinValue,DateTime.MinValue

//ICS related
let GetVEvents (c : Class) =
    let s = c.StartDateTime
    let e = c.EndDateTime
    //Fix the semantics: combine the time component of End with the date component of start to get the end time on a given day.
    let dayEndTime = new DateTime(s.Year,s.Month,s.Day,e.Hour,e.Minute,e.Second)
    seq {
        for day in c.Days do
            yield """BEGIN:VEVENT
DTSTART;TZID=America/Chicago:#START#
DTEND;TZID=America/Chicago:#END#
RRULE:FREQ=WEEKLY;UNTIL=#UNTILZ#;BYDAY=#DAY#
DTSTAMP:#NOWZ#
CREATED:#NOWZ#
DESCRIPTION:
LAST-MODIFIED:#NOWZ#
LOCATION:#WHERE#
SEQUENCE:0
STATUS:CONFIRMED
SUMMARY:#NAME#
TRANSP:OPAQUE
END:VEVENT""".Replace("#START#",c.StartDateTime.ToString("yyyyMMddTHHmmss")).Replace("#END#",dayEndTime.ToString("yyyyMMddTHHmmss")).Replace("#UNTILZ#",c.EndDateTime.ToUniversalTime().ToString("yyyyMMddTHHmmssZ")).Replace("#DAY#",day).Replace("#NOWZ#",DateTime.Now.ToUniversalTime().ToString("yyyyMMddTHHmmssZ")).Replace("#NAME#",c.Title).Replace("#WHERE#",c.Where)
    }

// Looked at https://github.com/UHCalendarTeam/ANtICalendar but it wasn't good
// Instead made a calendar template on Google and reverse engineered it
let WriteVEventsToFile filename vEvents = 
    let prefix = """BEGIN:VCALENDAR
    PRODID:-//Google Inc//Google Calendar 70.9054//EN
    VERSION:2.0
    CALSCALE:GREGORIAN
    METHOD:PUBLISH
    X-WR-CALNAME:#NAME#
    X-WR-TIMEZONE:America/Chicago
    BEGIN:VTIMEZONE
    TZID:America/Chicago
    X-LIC-LOCATION:America/Chicago
    BEGIN:DAYLIGHT
    TZOFFSETFROM:-0600
    TZOFFSETTO:-0500
    TZNAME:CDT
    DTSTART:19700308T020000
    RRULE:FREQ=YEARLY;BYMONTH=3;BYDAY=2SU
    END:DAYLIGHT
    BEGIN:STANDARD
    TZOFFSETFROM:-0500
    TZOFFSETTO:-0600
    TZNAME:CST
    DTSTART:19701101T020000
    RRULE:FREQ=YEARLY;BYMONTH=11;BYDAY=1SU
    END:STANDARD
    END:VTIMEZONE""".Replace("#NAME#",filename).Replace("    ","")
    let icsString = prefix + "\n" + (vEvents |> String.concat "\n") + "\n" + "END:VCALENDAR"
    System.IO.File.WriteAllText(filename + ".ics", icsString)


## Department Calendar View

Run the query to get all courses in a department, extract the time/date information for each course, then export this information in ICS format.
ICS can then be loaded into your favorite calendar app, e.g. Google Calendar. 
It is recommended to load each department ICS as its own calendar (rather than merging ICS into a single calendar) to retain color coding for departments.

In [34]:
//Banner Term, e.g. 201680 for Fall 2016; 201610 for Spring
let term = "201910"
let dept = "PHIL"

let classEvents =
    QueryDepartment dept term
    |> createDocAutoFix
    //Auto fix changes source html to HTML5 standard, so XPath must use e.g. Chrome inspector structure and not source structure
    |> selectNodes "html/body/div/table/tbody/tr/th/a | html/body/div/table/tbody/tr/td/table"
    //|> Seq.map( fun x -> x.InnerHtml) //for debug
    |> Seq.chunkBySize 2 //first is title; second is table of time/location/etc
    |> Seq.collect( fun courseChunk -> 
        //Need distinction b/w course and class chunks b/c Psychology has GenEd with variable classes across the week; see previous commit for simpler approach
        courseChunk.[1] 
        |> selectNodes "descendant::td" 
        |> Seq.chunkBySize 7
        |> Seq.map( fun classChunk -> 
            let days = GetDays classChunk.[2].InnerHtml
            let start,stop = GetStartEndDateTime classChunk.[4] classChunk.[1]
            {
                Title=courseChunk.[0].InnerHtml;
                Type=classChunk.[0].InnerHtml;
                Time=classChunk.[1].InnerHtml;
                StartDateTime =  start
                EndDateTime = stop
                Days=days;
                Where=classChunk.[3].InnerHtml;
                DateRange=classChunk.[4].InnerHtml;
                ScheduleType=classChunk.[5].InnerHtml;
                Instructors=classChunk.[6].InnerText;
                }
            )
      )
    //Purge events without times, e.g. dissertation and independent study
    |> Seq.filter (fun x -> x.StartDateTime <> DateTime.MinValue)
    |> Seq.collect GetVEvents
    |> Seq.toList
    
WriteVEventsToFile dept classEvents

"DONE"

"DONE"

## Class Calendar View

Given a list of courses, extract the time/date information for each course, then export this information in ICS format.

ICS can then be loaded into your favorite calendar app, e.g. Google Calendar. 

## Course List

The list of courses below was generated in a spreadsheet saved as tab delimited. 

In [37]:
let courseList = """degree	department	number
MINOR	PSYC	1030
MINOR	PSYC	3303
MINOR	PSYC	3530
MINOR	COMP	1900
MINOR	PHIL	3460
MINOR	PHIL	4421
MINOR	PHIL	4661
MINOR	ECON	4512
MINOR	COMP	2700
MINOR	COMP	4720
MINOR	MATH	2120
MINOR	MATH	4083
MINOR	ENGL	3511"""

In [39]:
let term = "201910"
let calendarName = "minor"

let classEvents = 
    courseList.Split("\n")
    |> Seq.skip 1 //skip header
    |> Seq.choose( fun x -> 
        let s = x.Split('\t')
        let degree = s.[0]
        let dept = s.[1]
        let id = s.[2]
        //handle queries that fail
        match QueryClass dept id term with
        | true, result -> Some( result )
        | false, _ -> None
        )
    |> Seq.map createDocAutoFix
    //|> Seq.map( fun x -> x.InnerHtml) //for debug
    |> Seq.collect( fun html -> 
        html
        //Auto fix changes source html to HTML5 standard, so XPath must use e.g. Chrome inspector structure and not source structure
        |> selectNodes "html/body/div/table/tbody/tr/th/a | html/body/div/table/tbody/tr/td/table"
        //|> Seq.map( fun x -> x.InnerHtml) //for debug
        |> Seq.chunkBySize 2 //first is title; second is table of time/location/etc
        |> Seq.collect( fun courseChunk -> 
            //Need distinction b/w course and class chunks b/c Psychology has GenEd with variable classes across the week; see previous commit for simpler approach
            courseChunk.[1] 
            |> selectNodes "descendant::td" 
            |> Seq.chunkBySize 7
            |> Seq.map( fun classChunk -> 
                let days = GetDays classChunk.[2].InnerHtml
                let start,stop = GetStartEndDateTime classChunk.[4] classChunk.[1]
                {
                    Title=courseChunk.[0].InnerHtml;
                    Type=classChunk.[0].InnerHtml;
                    Time=classChunk.[1].InnerHtml;
                    StartDateTime =  start
                    EndDateTime = stop
                    Days=days;
                    Where=classChunk.[3].InnerHtml;
                    DateRange=classChunk.[4].InnerHtml;
                    ScheduleType=classChunk.[5].InnerHtml;
                    Instructors=classChunk.[6].InnerText;
                    }
                )
          )
    )
    //Purge events without times, e.g. dissertation and independent study
    |> Seq.filter (fun x -> x.StartDateTime <> DateTime.MinValue)
    |> Seq.collect GetVEvents
    |> Seq.toList
    
WriteVEventsToFile calendarName classEvents
"DONE"

"DONE"